In [1]:
#%matplotlib inline
#import matplotlib.pyplot as plt
#plt.rcParams["figure.figsize"] = (11, 5)  #set default figure size

#from scipy.special import factorial

#from mpl_toolkits.mplot3d import Axes3D
#import statsmodels.api as sm
#from statsmodels.api import Poisson
#from scipy import stats
#from scipy.stats import norm
#from statsmodels.iolib.summary2 import summary_col

In [ ]:
# Packages and Data Load

In [41]:
import numpy as np
import pandas as pd
from numpy import exp
#import os
import ipywidgets as widgets
from IPython.display import display
import plotly_express as px

In [42]:
#df_tickets=pd.read_csv('/Users/danila.pankov/Downloads/glia.csv')
df_tickets=pd.read_csv('glia.csv')

In [43]:
#df_cleaned.head()

In [ ]:
# Data Adjustments

In [44]:
#source data cleaned
df=df_tickets.copy()
df["source_improved"]=df['source']
df.loc[df['source']=='tab', 'source_improved']='Glia'
df.loc[df['source']=='button_embed', 'source_improved']='Website'
df.loc[df['source']=='sdk', 'source_improved']='App'
df.loc[df['source']=='offline_phone', 'source_improved']='Phone'


df['queued_at'] = pd.to_datetime(df['queued_at'])

df['connected']=0
df.loc[df['outcome']=='finished', 'connected']=1

In [49]:
df_cleaned=df.copy()
df_cleaned=df_cleaned.drop(columns='source')
df_cleaned=df_cleaned.rename(columns={'source_improved': 'source'})

In [50]:
#print("Dropped data with nan values in the source variables")
df_cleaned=df_cleaned[df.source.notnull()].copy()

Dropped data with nan values in the source variables


In [94]:
def multiple_selection(df, variable):
    return widgets.SelectMultiple(
        options=df[variable].unique(),
        value=df[variable].unique().tolist(),
        #rows=10,
        description=variable,
        disabled=False
    )

In [110]:
#df_cleaned.queue_id.unique().tolist()

In [111]:
#type(["68058f1a9c69", '23825e66b683'])


In [97]:
queue=multiple_selection(df_cleaned, "queue_id" )
#queue

In [98]:
source=multiple_selection(df_cleaned, "source")
#source

In [55]:
#source.value

In [99]:
media=multiple_selection(df_cleaned, "requested_media_type")
#media

In [100]:
#outcome=multiple_selection(df_cleaned, "outcome", 'finished')
#outcome

In [101]:
#outcome.value

In [102]:
#df.outcome.isin(outcome.value)

In [103]:
freq='M' # or 'D' or 'Y'
df=df_cleaned.copy()
    
#print(df)
df_plot = df[['queued_at', 'connected','source']]\
        .groupby(['source',pd.Grouper(key='queued_at', freq=freq)])['connected'].agg(
        mean='mean', count='count').reset_index()
#df_plot

In [104]:
from datetime import datetime

start_date = df_cleaned.queued_at.dt.date.min()
end_date = df_cleaned.queued_at.dt.date.max()
#print(start_date)
dates = pd.date_range(start_date, end_date, freq='D')

options = [(date.strftime(' %d %b %Y '), date) for date in dates]
index = (0, len(options)-1)

selection_range_slider = widgets.SelectionRangeSlider(
    options=options,
    index=index,
    description='Dates',
    orientation='horizontal',
    layout={'width': '500px'}
)

#selection_range_slider

In [105]:
#df_cleaned

In [106]:
df_cleaned['hour_of_the_week'] = df_cleaned['queued_at'].dt.dayofweek * 24 + (df_cleaned['queued_at'].dt.hour + 1)

In [107]:
#df_cleaned[df_cleaned.queued_at.dt.date<df_cleaned.queued_at.dt.date.max()]

In [109]:
def update_map( queue, source, media):
    freq='H' # or 'D' or 'Y'
    df=df_cleaned.copy()
    
    #print(df)
    df_plot = df[df.queue_id.isin(queue) & 
                 #df.outcome.isin(outcome) & 
        df.requested_media_type.isin(media) & 
        df.source.isin(source) &
        (df.outcome!='visitor_conflict')][['hour_of_the_week', 'connected','source']]\
        .groupby(['source','hour_of_the_week'])['connected'].agg(
        mean='mean', count='count').reset_index()
    #print(df_plot)
    #fig = px.area(df_plot, x='queued_at', y='mean')
    #fig.show()
    fig = px.scatter(df_plot, x="hour_of_the_week", y="mean", color='source',
                 labels={
                     "hour_of_the_week": "Week Hour (0 - Monday 0:00, 167 - Sun 23:00)",
                     "count": "Percent of Solved Tickets",
                     "species": "Source Type"
                 },
                title="Percent of Solved Tickets over Time")
    fig.show()
    fig = px.scatter(df_plot, x="hour_of_the_week", y="count", color='source',
                 labels={
                     "hour_of_the_week": "Week Hour (0 - Monday 0:00, 167 - Sun 23:00)",
                     "count": "Total Number of Tickets",
                     "species": "Source Type"
                 },
                title="Number of Tickets over Time")
    fig.show()
    
widgets.interactive(update_map,queue=queue, source=source, media=media)

interactive(children=(SelectMultiple(description='queue_id', index=(0, 1, 2, 3), options=('68058f1a9c69', '238…

In [22]:
def update_map( queue, source, media,date_range):
    freq='H' # or 'D' or 'Y'
    df=df_cleaned.copy()
    
    #print(df)
    df_plot = df[df.queue_id.isin(queue) & 
                 #df.outcome.isin(outcome) & 
        df.requested_media_type.isin(media) & 
        df.source.isin(source) & 
        (df.queued_at.dt.date>= date_range[0]) & 
        (df.queued_at.dt.date< date_range[1])  &
        (df.outcome!='unstaffed')][['queued_at', 'connected','source']]\
        .groupby(['source',pd.Grouper(key='queued_at', freq=freq)])['connected'].agg(
        mean='mean', count='count').reset_index()
    #print(df_plot)
    #fig = px.area(df_plot, x='queued_at', y='mean')
    #fig.show()
    fig = px.line(df_plot, x="queued_at", y="mean", color='source')
    fig.show()
    fig = px.line(df_plot, x="queued_at", y="count", color='source')
    fig.show()
    
widgets.interactive(update_map,queue=queue, source=source, media=media, date_range=selection_range_slider)

interactive(children=(SelectMultiple(description='queue_id', index=(0,), options=('68058f1a9c69', '23825e66b68…